In [1]:
#Copyright 2020 Vraj Shah, Arun Kumar
#
#Licensed under the Apache License, Version 2.0 (the "License");
#you may not use this file except in compliance with the License.
#You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
#Unless required by applicable law or agreed to in writing, software
#distributed under the License is distributed on an "AS IS" BASIS,
#WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#See the License for the specific language governing permissions and
#limitations under the License.

import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import numpy as np

from Load_Predictions import *
from downstream_models import *
from Featurize import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# def get_col_dtype(col):
#         """
#         Identify datatype with pandas 
#         """
#         if col.dtype =="object":
#             try:
#                 col_new = pd.to_datetime(col)
#                 return col_new.dtype
#             except:
#                 try:
#                     col_new = pd.to_numeric(col)
#                     print(col_new.dtype)
#                     return col_new.dtype
#                 except:
#                     return "Object"
#         if col.dtype =="bool": return "Object"
#         else:
#             if col.dtype == 'float64' or col.dtype == 'int64':
#                 return 'Numeric'
#             else:      
#                 return col.dtype

In [3]:
testdf = pd.read_csv('Benchmark-Labeled-Data/data_test.csv')
test_metadata = pd.read_csv('../RawCSV/Metadata/meta_data.csv')

print(len(testdf),len(test_metadata))
test_merged = pd.merge(testdf,test_metadata,on='Record_id')
print(len(test_merged))

print(test_merged)
y_true = test_merged.y_act.values.tolist()
# print(y_true)
print(len(y_true))

1985 4233
1985
      Record_id                                  Attribute_name  \
0           285                     Net Income-Cont. Operations   
1           285                Earnings Before Interest and Tax   
2           285                                   After Tax ROE   
3           285                                        For Year   
4           285  Equity Earnings/Loss Unconsolidated Subsidiary   
...         ...                                             ...   
1980        328                                            item   
1981        343                                     load_weight   
1982       1970                                    SpecialNotes   
1983       2050                                         Website   
1984        222                                             NaN   

                  y_act  total_vals  num_nans     %_nans  num_of_dist_val  \
0               numeric        1781         0   0.000000             1693   
1               numeric   

In [4]:
dict_label_true = {
    'numeric': 0,
    'categorical': 1,
    'datetime': 2,
    'sentence': 3,
    'url': 4,
    'embedded-number': 5,
    'list': 6,
    'not-generalizable': 7,
    'context-specific': 8
}

y_true = [dict_label_true[str(i)] for i in y_true]

In [5]:
sample_size_list = [1,2,3,4,5,10]
rand_seed = 99

final_results = []

for sample_size in sample_size_list:
    y_pandas = []
    prv_csv_name,csv_name = '',''
    exception_indices = []
    error_count = 0

    for index, row in test_merged.iterrows():     
        if index%10==0:  print(index)
        col = row['Attribute_name']
        prv_csv_name = csv_name
        csv_name = '../RawCSV/RawCSVFiles/' + row['name']

#         print(csv_name)
#         print(col)

        if prv_csv_name != csv_name:  df = pd.read_csv(csv_name,encoding='latin1')

        try:
            df_sample = df[[col]].sample(n=sample_size, random_state=rand_seed)
            dataFeaturized = FeaturizeFile(df_sample)

            dataFeaturized1 = ProcessStats(dataFeaturized)
            dataFeaturized2 = FeatureExtraction(dataFeaturized,dataFeaturized1,0)
            dataFeaturized2 = dataFeaturized2.fillna(0)

            y_RF = Load_RF(dataFeaturized2)
#             print(y_RF)

            y_pandas.append(int(y_RF[0]))
        except:
            error_count += 1
            y_pandas.append(7)
            
    results = pd.DataFrame(
    {'actual': y_true,
     'predicted': y_pandas
    })
    
    cmat = confusion_matrix(results['actual'], results['predicted'])
    accuracies = []

    for key, value in dict_label_true.items(): 
        TP=0
        FP=0
        TN=0
        FN=0
        for i in range(0,9):
            for j in range(0,9):
                total_actual = sum(cmat[i])
                if (i == value & i == j):
                    TP = cmat[i][j]
                    FP = total_actual-TP
                if i != value:
                    TN += cmat[i][j]
                if (i != value) & (j == value):
                    FN += cmat[i][j]

        print(TP, FP, TN, FN)
        accuracy = (TP+(TN-FN))/(TP+FP+FN+(TN-FN))
        print(accuracy)
        accuracies.append(accuracy)
        
    class_accuracy = list(zip(dict_label_true.keys(), accuracies))
    class_accuracy = pd.DataFrame(class_accuracy).set_index(0).T
    class_accuracy.index = ['accuracy']
    
    classification_report_df = pd.DataFrame(classification_report(results['actual'], results['predicted'], target_names=dict_label_true.keys(), output_dict=True))
    classification_report_df = classification_report_df.loc[['precision', 'recall', 'f1-score']].iloc[: , :-3]
    
    result_df = class_accuracy.append(classification_report_df)
    final_results.append([result_df, cmat])

0
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
10
1 1
1 1
1 1


C:\Users\Andrew\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (41,142) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1 1
1 1
1 1
1 1
1 1
1 1
1 1
20
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
30
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
40
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
50
1 1
1 1
1 1
1 1


C:\Users\Andrew\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (40,141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1 1
1 1
1 1
1 1
1 1
1 1
60
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
70
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
80
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
90
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
100
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
110
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
120
1 1


...

1 1
1850
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1860
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1870
1 1
1 1
1 1
1 1
1 1
1 1
1 1


C:\Users\Andrew\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (5,7,8,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1 1
1 1
1 1
1880
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1890
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1900
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1910
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1920
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1930
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1940
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1950
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1960
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1970
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1


C:\Users\Andrew\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1 1
1 1
1980
1 1
1 1
1 1
1 1
618 89 1278 82
0.9138539042821159
298 159 1528 61
0.889168765743073
121 20 1844 6
0.9869017632241813
68 24 1893 23
0.9763224181360202
26 6 1953 0
0.996977329974811
71 28 1886 13
0.979345088161209
4 53 1928 4
0.9712846347607053
192 23 1770 361
0.8065491183879093
30 155 1800 7
0.9183879093198992


In [6]:
final_results[5][1]

array([[618,  13,   1,   0,   0,   0,   0,  75,   0],
       [ 23, 298,   0,  16,   0,   2,   2, 112,   4],
       [  0,   1, 121,   0,   0,   2,   0,  17,   0],
       [  0,   1,   0,  68,   0,   1,   1,  21,   0],
       [  0,   0,   1,   1,  26,   0,   0,   4,   0],
       [  0,   6,   1,   0,   0,  71,   1,  20,   0],
       [  1,   2,   0,   2,   0,   5,   4,  41,   2],
       [  2,  16,   2,   1,   0,   1,   0, 192,   1],
       [ 56,  22,   1,   3,   0,   2,   0,  71,  30]], dtype=int64)

In [7]:
sample_size_results_list = []

for i in range(len(sample_size_list)):
    results_df = final_results[i][0]
    
    results_df = pd.DataFrame(results_df.T.stack()).rename({0: f"Sameple Size: {sample_size_list[i]}"}, axis=1)
    results_df.index = results_df.index.set_names(['Feature Type', 'Metric'])
    
    sample_size_results_list.append(results_df)

In [8]:
benchmark_df = pd.concat(sample_size_results_list, axis=1)
benchmark_df

Sameple Size: 1  Sameple Size: 2  \
Feature Type      Metric                                        
numeric           accuracy          0.791436         0.887657   
                  precision         0.974110         0.921603   
                  recall            0.425743         0.748232   
                  f1-score          0.592520         0.825917   
categorical       accuracy          0.783879         0.836272   
                  precision         0.937500         0.748120   
                  recall            0.065646         0.435449   
                  f1-score          0.122699         0.550484   
datetime          accuracy          0.983879         0.988413   
                  precision         0.957983         0.946970   
                  recall            0.808511         0.886525   
                  f1-score          0.876923         0.915751   
sentence          accuracy          0.956675         0.976826   
                  precision         0.875000         0.828571   
                  recall            0.076087         0.630435   
                  f1-score          0.140000         0.716049   
url               accuracy          0.995466         0.996474   
                  precision         0.960000         0.962963   
                  recall            0.750000         0.812500   
                  f1-score          0.842105         0.881356   
embedded-number   accuracy          0.980856         0.980856   
                  precision         0.955224         0.929577   
                  recall            0.646465         0.666667   
                  f1-score          0.771084         0.776471   
list              accuracy          0.973804         0.977834   
                  precision         0.528090         0.580247   
                  recall            0.824561         0.824561   
                  f1-score          0.643836         0.681159   
not-generalizable accuracy          0.431234         0.701259   
                  precision         0.156907         0.248670   
                  recall            0.972093         0.869767   
                  f1-score          0.270200         0.386763   
context-specific  accuracy          0.908816         0.909824   
                  precision         1.000000         0.750000   
                  recall            0.021622         0.048649   
                  f1-score          0.042328         0.091371   

                             Sameple Size: 3  Sameple Size: 4  \
Feature Type      Metric                                        
numeric           accuracy          0.895718         0.906801   
                  precision         0.913907         0.912975   
                  recall            0.780764         0.816124   
                  f1-score          0.842105         0.861837   
categorical       accuracy          0.857935         0.865491   
                  precision         0.788779         0.787879   
                  recall            0.522976         0.568928   
                  f1-score          0.628947         0.660737   
datetime          accuracy          0.990428         0.991436   
                  precision         0.962121         0.962687   
                  recall            0.900709         0.914894   
                  f1-score          0.930403         0.938182   
sentence          accuracy          0.978338         0.978338   
                  precision         0.802469         0.781609   
                  recall            0.706522         0.739130   
                  f1-score          0.751445         0.759777   
url               accuracy          0.995970         0.995970   
                  precision         0.961538         0.961538   
                  recall            0.781250         0.781250   
                  f1-score          0.862069         0.862069   
embedded-number   accuracy          0.983375         0.980856   
                  precision         0.892857         0.876543  

In [9]:
benchmark_df.to_latex('Sample Size Benchmark.tex')

In [19]:
cmat_10 = pd.DataFrame(final_results[5][1])
label_dict = {v: k for k, v in dict_label_true.items()}
cmat_df = cmat_10
cmat_df = cmat_df.rename(label_dict)
cmat_df = cmat_df.rename(columns=label_dict)
cmat_df.to_latex("10_sample_cmat.tex")
cmat_df

,numeric,categorical,datetime,sentence,url,embedded-number,list,not-generalizable,context-specific
numeric,618,13,1,0,0,0,0,75,0
categorical,23,298,0,16,0,2,2,112,4
datetime,0,1,121,0,0,2,0,17,0
sentence,0,1,0,68,0,1,1,21,0
url,0,0,1,1,26,0,0,4,0
embedded-number,0,6,1,0,0,71,1,20,0
list,1,2,0,2,0,5,4,41,2
not-generalizable,2,16,2,1,0,1,0,192,1
context-specific,56,22,1,3,0,2,0,71,30


In [20]:
cmat_5 = pd.DataFrame(final_results[4][1])
label_dict = {v: k for k, v in dict_label_true.items()}
cmat_df = cmat_5
cmat_df = cmat_df.rename(label_dict)
cmat_df = cmat_df.rename(columns=label_dict)
cmat_df.to_latex("5_sample_cmat.tex")
cmat_df

,numeric,categorical,datetime,sentence,url,embedded-number,list,not-generalizable,context-specific
numeric,604,9,1,0,0,0,0,93,0
categorical,15,269,1,16,0,1,2,150,3
datetime,0,2,130,0,0,2,0,7,0
sentence,0,2,0,69,0,0,1,20,0
url,0,0,0,1,27,0,0,4,0
embedded-number,0,6,1,0,0,71,1,20,0
list,1,2,0,2,0,3,7,41,1
not-generalizable,4,20,1,1,1,1,0,186,1
context-specific,44,20,1,1,0,2,2,89,26


In [21]:
cmat_5 = pd.DataFrame(final_results[0][1])
label_dict = {v: k for k, v in dict_label_true.items()}
cmat_df = cmat_5
cmat_df = cmat_df.rename(label_dict)
cmat_df = cmat_df.rename(columns=label_dict)
cmat_df.to_latex("1_sample_cmat.tex")
cmat_df

,numeric,categorical,datetime,sentence,url,embedded-number,list,not-generalizable,context-specific
numeric,301,0,0,0,0,0,0,406,0
categorical,0,30,3,1,0,1,10,412,0
datetime,0,0,114,0,0,0,3,24,0
sentence,0,0,0,7,0,0,15,70,0
url,0,0,0,0,24,0,1,7,0
embedded-number,0,0,1,0,0,64,7,27,0
list,0,0,0,0,0,2,47,8,0
not-generalizable,1,2,1,0,1,0,1,209,0
context-specific,7,0,0,0,0,0,5,169,4
